**Goal.** Use lightweight sentiment analysis on transcripts to triage support calls.

| What               | Why                                   | Outcome                       |
| ------------------ | ------------------------------------- | ----------------------------- |
| Sentiment analysis | Detect negative/neutral/positive tone | Prioritize problematic calls  |
| VADER (NLTK)       | Pretrained, rule-lexicon hybrid       | Fast, no training data needed |

**Setup.** Install NLTK and fetch required models.

| Step          | Command                                                                       | Note                          |
| ------------- | ----------------------------------------------------------------------------- | ----------------------------- |
| Install       | `pip install nltk`                                                            | Once per environment          |
| Download data | Python: `import nltk; nltk.download('punkt'); nltk.download('vader_lexicon')` | Tokenizer + sentiment lexicon |

**VADER outputs.** What the analyzer returns and how to read it.

| Key        | Meaning                  |  Range | Interpretation                       |
| ---------- | ------------------------ | -----: | ------------------------------------ |
| `neg`      | Negative proportion      |    0–1 | Higher ⇒ more negative tokens        |
| `neu`      | Neutral proportion       |    0–1 | Middle mass                          |
| `pos`      | Positive proportion      |    0–1 | Higher ⇒ more positive tokens        |
| `compound` | Normalized overall score | −1..+1 | ≲−0.05 neg, ≳+0.05 pos, else neutral |

**Minimal code.** Initialize VADER and score a transcript string.


In [ ]:

# NLTK VADER: single-pass sentiment on a transcript string
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# one-time per environment:
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
text = "I still haven't received my product. The service is terrible."
scores = sid.polarity_scores(text)
print(scores)  # {'neg': ..., 'neu': ..., 'pos': ..., 'compound': ...}




**Sentence-level analysis.** Sentiment shifts inside a call, so score per sentence.

| Reason                   | Method                           | Benefit                      |
| ------------------------ | -------------------------------- | ---------------------------- |
| Tone varies by utterance | Split into sentences, score each | Pinpoint troublesome moments |


In [ ]:

# Sentence-by-sentence scoring
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

# one-time:
# nltk.download('punkt'); nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
transcript = "I still haven't received my product. The service is terrible. Thanks for the update."

rows = []
for s in sent_tokenize(transcript):
    sc = sid.polarity_scores(s)
    rows.append((s, sc["neg"], sc["neu"], sc["pos"], sc["compound"]))

for r in rows:
    print(r)



**ASR → sentiment caveats.** Transcription quality affects scores.

| Issue                     | Effect on sentiment | Mitigation                                        |
| ------------------------- | ------------------- | ------------------------------------------------- |
| Missing negations (“not”) | Flips polarity      | Prefer higher-quality ASR; manual QA for outliers |
| Disfluencies/fillers      | Inflate neutrality  | Clean text or ignore fillers                      |
| Domain terms              | Unseen by lexicon   | Add custom lexicon or rules                       |

**Pipeline.** Practical flow from audio to actionable sentiment.

| Stage      | Tool                                   | Output                             |
| ---------- | -------------------------------------- | ---------------------------------- |
| Transcribe | Your ASR (free or paid)                | Text or sentences                  |
| Score      | VADER on whole text and per sentence   | Compound and token proportions     |
| Act        | Thresholds on compound; flag negatives | Review segments with lowest scores |

**Heuristic thresholds.** Use simple rules for triage.

| Rule                      | Action           |
| ------------------------- | ---------------- |
| `compound ≤ −0.05`        | Flag as negative |
| `−0.05 < compound < 0.05` | Mark neutral     |
| `compound ≥ 0.05`         | Mark positive    |
